In [1]:
DEBUG = False

In [7]:
from fastai.vision.all import *
from config import WANDB_PROJECT, BDD_CLASSES, ENTITY, RAW_DATA_AT
import wandb

In [3]:
URL = 'https://storage.googleapis.com/wandb_course/bdd_sample_1k.zip'

In [4]:
path = Path(untar_data(URL))

In [5]:
path.ls()

(#3) [Path('/home/darek/.fastai/data/bdd_sample_1k/labels'),Path('/home/darek/.fastai/data/bdd_sample_1k/LICENSE.txt'),Path('/home/darek/.fastai/data/bdd_sample_1k/images')]

In [6]:
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

def get_classes_per_image(mask_data, class_labels):
    unique = list(np.unique(mask_data))
    result_dict = {}
    for _class in class_labels.keys():
        result_dict[class_labels[_class]] = int(_class in unique)
    return result_dict

def _create_table(image_files, class_labels):
    "Create a table with the dataset"
    labels = [str(class_labels[_lab]) for _lab in list(class_labels)]
    table = wandb.Table(columns=["File_Name", "Images", "Split"] + labels)
    
    for i, image_file in progress_bar(enumerate(image_files), total=len(image_files)):
        image = Image.open(image_file)
        mask_data = np.array(Image.open(label_func(image_file)))
        class_in_image = get_classes_per_image(mask_data, BDD_CLASSES)
        table.add_data(
            str(image_file.name),
            wandb.Image(
                    image,
                    masks={
                        "predictions": {
                            "mask_data": mask_data,
                            "class_labels": class_labels,
                        }
                    }
            ),
            "None", # we don't have a dataset split yet
            *[class_in_image[_lab] for _lab in labels]
        )
    
    return table

In [8]:
run = wandb.init(project=WANDB_PROJECT, entity=ENTITY, job_type="upload")
raw_data_at = wandb.Artifact(RAW_DATA_AT, type="raw_data")

wandb: Currently logged in as: darek. Use `wandb login --relogin` to force relogin


In [9]:
raw_data_at.add_file(path/'LICENSE.txt', name='LICENSE.txt')

<ManifestEntry digest: X+6ZFkDOlnKesJCNt20yRg==>

In [10]:
raw_data_at.add_dir(path/'images', name='images')
raw_data_at.add_dir(path/'labels', name='labels')

wandb: Adding directory to artifact (/home/darek/.fastai/data/bdd_sample_1k/images)... Done. 0.1s
wandb: Adding directory to artifact (/home/darek/.fastai/data/bdd_sample_1k/labels)... Done. 0.1s


In [11]:
image_files = get_image_files(path/"images", recurse=False)
if DEBUG: image_files = image_files[:10]

In [12]:
table = _create_table(image_files, BDD_CLASSES)

In [13]:
raw_data_at.add(table, "eda_table")

<ManifestEntry digest: /WLW1gUqf8H5IoaqMqLUhg==>

In [14]:
run.log_artifact(raw_data_at)
run.finish()